# Machine Learning Notebook for bike/parking space predictions

## Section 0: Import Required Libraries

In [26]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pickle

## Section 1: Data preparation

### Section 1.1: Create a function to create to the SQL database

In [2]:
def connect_to_database():
    """Function for connecting to the SQL database"""

    # Create variables to store cretentials
    USER = "admin"
    DB = "dbikes"
    PORT = "3306"
    URL = "database-1.ctesjcult8dm.eu-west-1.rds.amazonaws.com"

    # Read in password from text file
    with open('../mysql_password.txt') as f:
        PASSWORD = ''.join(f.readlines())
        PASSWORD = str(PASSWORD).split()[0]

    # Create engine using credentials
    engine = create_engine(
        "mysql+mysqlconnector://{}:{}@{}:{}/{}".format(USER, PASSWORD, URL, PORT, DB), echo=True)

    # Create connection using engine and return connection
    conn = engine.connect()
    return conn


In [3]:
engine = connect_to_database()

2022-03-23 17:12:40,769 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-03-23 17:12:40,772 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 17:12:40,793 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-03-23 17:12:40,797 INFO sqlalchemy.engine.Engine [generated in 0.00709s] {}
2022-03-23 17:12:40,830 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-03-23 17:12:40,832 INFO sqlalchemy.engine.Engine [raw sql] {}


### Section 1.2: Read in real time bike/parking space availability data

In [4]:
availability_df = pd.read_sql_query("SELECT * FROM dbikes.availability", engine)

2022-03-23 17:12:40,963 INFO sqlalchemy.engine.Engine SELECT * FROM dbikes.availability
2022-03-23 17:12:40,965 INFO sqlalchemy.engine.Engine [raw sql] {}


### Section 1.3: Remove Duplicate values from real time availability

In [5]:
# Remove duplicates
availability_df.drop_duplicates(inplace=True)

### Section 1.4: Read in weather data

In [6]:
weather_df = pd.read_sql_query("SELECT dt, temperature, main FROM dbikes.real_time_weather", engine)

2022-03-23 17:13:02,446 INFO sqlalchemy.engine.Engine SELECT dt, temperature, main FROM dbikes.real_time_weather
2022-03-23 17:13:02,447 INFO sqlalchemy.engine.Engine [raw sql] {}


### Section 1.5: Remove duplicate values from weather

In [7]:
#Drop duplicates
weather_df.drop_duplicates(inplace=True)

### Section 1.6: Rename last update column in availability dataframe to dt to help with the merging of the availability and weather dataframes

In [8]:
#Changes the name of the column
availability_df.rename(columns={"last_update": "dt"}, inplace=True)
availability_df.columns

Index(['number', 'available_bikes', 'available_stands', 'dt'], dtype='object')

### Section 1.7: Change the type of the dt variable to an np.int64 type for merging

In [9]:
availability_df["dt"] = availability_df["dt"].astype(np.int64)
weather_df["dt"]= weather_df["dt"].astype(np.int64)

### Section 1.8: Sort the dt columns for merging

In [10]:
availability_df.sort_values(by="dt", inplace=True)
weather_df.sort_values(by="dt", inplace=True)

### Section 1.9: Convert seconds to milliseconds in dt column in availability so that it is comparable with the corresponding weather column

In [11]:
#Converts seconds to miliseconds
availability_df["dt"] = availability_df["dt"] // 1000

### Section 1.10: Merge weather and availability dataframes

In [12]:
df=pd.merge_asof(availability_df, weather_df, on="dt", direction="nearest")

### Section 1.11: Convert temperature from Kelvin to Celcius

In [13]:
df["temperature"] = df["temperature"] - 273.15

### Section 1.12: Transform the dt column to get hour of the day and day of the week

In [14]:
df["dt"] = pd.to_datetime(df["dt"], unit="s")
df["day_of_week"] = df["dt"].dt.day_name()
df["hour"] = df["dt"].dt.hour

### Section 1.13: Convert categorical variables to "category" data type

In [15]:
df["main"] = df["main"].astype("category")
df["day_of_week"] = df["day_of_week"].astype("category")
df["hour"] = df["hour"].astype("category")

### Section 1.14: Get a list of station numbers

In [16]:
station_numbers = list(df["number"].unique())

### Section 1.15: Reencode categorical variables using binary encoding

In [21]:
dummy_fields = ["main", "hour", "day_of_week"]
for each in dummy_fields:
    dummies = pd.get_dummies(df[each], prefix=each, drop_first = False)
    df = pd.concat([df, dummies], axis=1)
df = df.drop(dummy_fields, axis=1)

In [24]:
x_column_names = list(df.columns)[4:]

['temperature', 'main_Clear', 'main_Clouds', 'main_Drizzle', 'main_Mist', 'main_Rain', 'main_Snow', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday']


## Section 2: Model creation/evaluation

In [27]:
model={}

for station in [2, 3]:
    station_df = df[df["number"] == station]
    X = station_df[x_column_names]
    y = station_df["available_bikes"]
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model[station] = LogisticRegression( solver='lbfgs', max_iter=1500)
    model[station].fit(X_train, y_train)
    file_name = "station_" + str(station) + "_bike_model.pkl"
    with open(file_name, 'wb') as handle:
        pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)
    print(station)

2
3
{2: LogisticRegression(max_iter=1500), 3: LogisticRegression(max_iter=1500)}


In [38]:
sample_x_data = X_test.head(1)
sample_x_data

,temperature,main_Clear,main_Clouds,main_Drizzle,main_Mist,main_Rain,main_Snow,hour_0,hour_1,hour_2,...,hour_21,hour_22,hour_23,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
689846,7.85,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [39]:
with open('station_2_bike_model.pkl', 'rb') as handle:
    station_2_model = pickle.load(handle) 

In [43]:
print(station_2_model[2].predict(sample_x_data))

[10]
